# Context-sensitive Spelling Correction

### by Danila Shulepin (BS21-DS-02)

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

## Implementation of context-sensitive spelling correction

Downloading and importing packages

In [163]:
# !pip install annoy
# !pip install gdown
# # !pip freeze > requirements.txt

In [143]:
import numpy as np
import gdown
import string
from tqdm import tqdm
from annoy import AnnoyIndex
from nltk import edit_distance
from urllib.request import urlretrieve
from random import random, sample, choice, shuffle
from nltk.tokenize import RegexpTokenizer

Downloading bigrams and fivegrams for ngram model.

In [144]:
url = 'https://drive.google.com/uc?id=1LauUKpATAXc6rvY5xKhS7rFWoJRxYt93'
output = 'bigrams.txt'
gdown.download(url, output, quiet=False)

bigrams = {}
unigrmas = {}
for a in open('./bigrams.txt', 'r', encoding = "ISO-8859-1").readlines():
    b, c, d = a.strip().split()
    bigrams[(c, d)] = int(b)
    if tuple([c]) not in unigrmas.keys():
        unigrmas[tuple([c])] = int(b)
    else:
        unigrmas[tuple([c])] += int(b)

Downloading...
From: https://drive.google.com/uc?id=1LauUKpATAXc6rvY5xKhS7rFWoJRxYt93
To: /content/bigrams.txt
100%|██████████| 17.7M/17.7M [00:00<00:00, 55.5MB/s]


In [145]:
url = 'https://drive.google.com/uc?id=1N7uOUeP8rPnLfXSoJV4L4PN-R1uh-bd9'
output = '5gram.txt'
gdown.download(url, output, quiet=False)

fivegram = {}
for a in open('./5gram.txt', 'r', encoding = "ISO-8859-1").readlines():
    b, c, d, e, f, g = a.strip().split()
    fivegram[(c, d, e, f, g)] = int(b)

Downloading...
From: https://drive.google.com/uc?id=1N7uOUeP8rPnLfXSoJV4L4PN-R1uh-bd9
To: /content/5gram.txt
100%|██████████| 28.3M/28.3M [00:00<00:00, 79.2MB/s]


Downloading Norvig unigrams [1] with word frequency and building vocab based on this set.

In [146]:
url = "https://norvig.com/ngrams/count_1w.txt"
filename = "norvig.txt"
urlretrieve(url, filename)

('norvig.txt', <http.client.HTTPMessage at 0x7c3a5d78b640>)

In [147]:
WORDS = set([a.strip().split()[0]
             for a in open(filename, 'r',
                           encoding = "ISO-8859-1").readlines()][:15000])

Building classes for my custom SpellChecker and Norvig solution

In [148]:
class SpellChecker:
    # Custom spellchecker
    def __init__(self, WORDS, ngrams, freq):
        """
        Initializing vocabulars of the spellchecker

        :param WORDS: frequent words vocabular
        :param ngram: list of ngram frequencies
        :param freq: frequencies of word unigrams
        """
        self.WORDS = WORDS
        self.letters = string.ascii_lowercase
        self.ngrams = ngrams
        self.freq = freq

        self.dim = len(self.letters) + 1
        self.embeddings = np.zeros((len(WORDS), self.dim), dtype=np.float16)
        self.text_embedding() # Running embedding of each word in vocab
        self.embedding_index = self.get_vector_index() # Get Annoy index

        self.tokenizer = RegexpTokenizer(r'\w+') # Tokenizer

    def embed(self, text):
        """
        Word embedding based on symbol frequencies in words

        :param text: string word
        :return: vector of embedding
        """
        vect = []
        base = {i:0 for i in self.letters}

        # Add unknown token
        base['<UNK>'] = 0
        for i in list(text.lower()):
            # Count each symbol in words
            if i in base.keys():
                base[i] += 1
            else:
                base['<UNK>'] += 1
        vect = list(base.values())
        return vect

    def text_embedding(self):
        """
        Embed all words in vocab
        """
        print("Running embedding...")
        for i, item in enumerate(tqdm(self.WORDS)):
            # Embed each word of vocab
            emb = self.embed(item)
            if emb is not None:
                self.embeddings[i, :] = emb
        print("Embeddig finished!\n")

    def get_vector_index(self):
        """
        Creating Annoy index based on embedded data

        :return: Annoy index
        """
        # Use manhattan distance for measuring similarity
        idx = AnnoyIndex(self.dim, 'manhattan')
        idx.set_seed(33)
        print("Building an index...")
        for i in tqdm(range(self.embeddings.shape[0])):
            idx.add_item(i, self.embeddings[i])
        print("Building trees...")
        idx.build(37)
        idx.save('./my_annoy.ann')
        print("Finished!")
        return idx

    def get_kNN_embeddings(self, embedding, k, index):
        """
        Creating Annoy index based on embedded data

        :param embedding: vector of embedded text
        :param k: number of neighbors
        :param index: Annoy index
        :return: ids of neighbors
        """
        return index.get_nns_by_vector(embedding, k)

    def get_top_k(self, query, idx_k = 150, idx_top = 70, max_k = 15, min_d = 2):
        """
        Get top of best candidates for replacement

        :param query: word with potential mistake
        :param idx_k: k neighbors by Annoy
        :param idx_top: limit Annoy neighbors
        :param max_k: limit edit distance neighbors
        :param min_d: minimal acceptable distance
        :return: top words
        """
        if len(query) != 0:
            # Get indexes of nearest neighbors
            idx = self.get_kNN_embeddings(self.embed(query), 150,
                                          self.embedding_index)[:70]
            d = {}
            for i in range(len(idx)):
                # Measure distances between word and potential candidates
                word = list(self.WORDS)[idx[i]]
                d[word] = edit_distance(query, word)
            # Get words with shortest edit distance
            top_words = [x for x, y in sorted(d.items(), key=lambda x:x[1]) if y <= min_d][:max_k]
            return top_words
        return ['']

    def get_neighbors(self, idx, tokens, n):
        """
        Get ngrams from sentence with specified word

        :param idx: id of word in tokens list
        :param tokens: list of sentence tokens
        :param n: size of ngram
        :return: list of ngrams
        """
        neighbors = []
        for i in range(idx - n + 1, idx + 1):
            if i < 0 or i + n > len(tokens):
                continue
            else:
                part = [x.lower() for x in tokens[i:i+n]]
                neighbors.append(part)
        if len(neighbors) != 0:
            return neighbors
        else:
            return [[x.lower()] for x in tokens]

    def get_probability(self, ngram):
        """
        Get probability of geting such a ngram: P(A|B) = P(A,B)/P(B)

        :param ngram: ngram of words
        :return: probability of getting word, seen words before
        """
        t = tuple(ngram)
        if t in self.ngrams.keys():
            if tuple(ngram[:-1]) in self.freq.keys():
                # Smoothing
                return (int(self.ngrams[t])+1)/(int(self.freq[tuple(ngram[:-1])]) + len(self.freq))
        return 1/len(self.freq)

    def perplexity(self, probabilities):
        """
        Get perplexity: PP = (1/P_1*...P_i*...*P_n)^(1/n)

        :param probabilities: list of probabilities
        :return: value of perplexity
        """
        return (1/np.prod(probabilities))**(1/len(probabilities))

    def get_probabilities(self, true_word, top_words, neighbors):
        """
        Get perplexities for list of top words

        :param true_word: candidate word for replacement
        :param top_words:
        """
        perps = []
        for word in top_words:
            ps = []
            for ngram in neighbors:
                idx = ngram.index(true_word.lower())
                new_ngram = ngram[:idx] + [word] + ngram[idx+1:]
                # Searching for probability of such ngram with word replaced by candidate
                p = self.get_probability(new_ngram)
                ps.append(p)
            # Calculate perplexities
            perp = self.perplexity(ps)
            perps.append(perp)
        return perps

    def spell_check(self, sentence, n = 2):
        """
        SpellChecking procedure

        :param sentence: sentence to spellcheck
        :param n: size of ngram
        :return: changed sentence
        """
        sent = sentence
        # Tokenizing sentence
        tokens = self.tokenizer.tokenize(sentence)
        for i, word in enumerate(tokens):
            if word.lower() in self.WORDS:
                # Skip words from vocab
                continue
            else:
                # Get best candidates
                top_words = self.get_top_k(word, min_d = 2)
                if len(top_words) == 0:
                    # The word is unknown
                    continue
                else:
                    # Find best replacement
                    best_perplexity = -1
                    best_word = ""
                    neighbors = self.get_neighbors(i, tokens, n)
                    probs = self.get_probabilities(word, top_words, neighbors)
                    for j, prob in enumerate(probs):
                        if prob < best_perplexity or best_perplexity == -1:
                            best_word = top_words[j]
                            best_perplexity = prob
                    sent = sent.replace(word, best_word)
                    idx = tokens.index(word)
                    tokens = tokens[:idx] + [best_word] + tokens[idx+1:]
        return sent
sc = SpellChecker(WORDS, bigrams, unigrmas)

Running embedding...


100%|██████████| 15000/15000 [00:00<00:00, 127353.03it/s]


Embeddig finished!

Building an index...


100%|██████████| 15000/15000 [00:00<00:00, 216410.21it/s]


Building trees...
Finished!


In [161]:
sentence_1 = "I like dking sport in the morning."
sentence_2 = "Only clever dking rules the country."

print(sc.spell_check(sentence_1))
print(sc.spell_check(sentence_2))

I like doing sport in the morning.
Only clever king rules the country.


In [ ]:
import re
from collections import Counter

# Norvig solution from [3]
class Norvig:
    def __init__(self):
        url = "https://norvig.com/big.txt"
        filename = "big.txt"
        urlretrieve(url, filename)
        self.WORDS = Counter(self.words(open('big.txt').read()))

    def words(self, text): return re.findall(r'\w+', text.lower())

    def P(self, word, N = None):
        if N is None:
            N = sum(self.WORDS.values())
        return self.WORDS[word] / N

    def correction(self, word):
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word):
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

    def known(self, words):
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word):
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

    def full_correct(self, sentence):
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(sentence)
        for token in tokens:
            correct_token = self.correction(token)
            sentence = sentence.replace(token, correct_token)
        return sentence

norvig = Norvig()

## Justify your decisions

The following ideas were used to create a custom spellchecker:
1. If a word exists in the vocabulary, then it is spelled correctly and does not need corrections;

2. If the word does not exist in the vocabulary, there are two possible outcomes:
         - The word in the vocabulary that is misspelled;
         - A new word that does not exist in the vocabulary at all.

3. In order to find out, one can check if it is possible to get a word from the dictionary by a few corrections. This suggests measuring the edit distance between the misspelled word and potential candidates for correction [2]. However, edit distance is a relatively long operation, so computing it over the entire vocabulary might be too long to check a single word. For this reason, I had the idea to use a faster search option - indexing. I chose Annoy as such a tool.

4. Annoy [3] (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings to search for points in space that are close to a given query point. Annoy is using random projections and by building up a tree. At every intermediate node in the tree, a random hyperplane is chosen, which divides the space into two subspaces. This hyperplane is chosen by sampling two points from the subset and taking the hyperplane equidistant from them. We do this k times so that we get a forest of trees. k has to be tuned to your need, by looking at what tradeoff you have between precision and performance. Annoy is almost as fast as the fastest libraries, and it has the ability to use static files as indexes. In particular, this means you can share index across processes. Annoy also decouples creating indexes from loading them, so you can pass around indexes as files and map them into memory quickly. Another nice thing of Annoy is that it tries to minimize memory footprint so the indexes are quite small.

5. To build Annoy trees one needs to map words into some n-dimensional space, in other words, to do embedding. I thought it might make sense to encode words through the frequency of the letters they contain. This way each word would correspond to a (26 + 1)-dimensional vector. The +1 is added to keep track of unknown characters (non-letters) that might occur in the words. For such embedding, it may also be reasonable to train Annoy using Manhattan distance.

6. Now you can quickly select candidates with a similar set of letters for further analysis. Among the words obtained with Annoy, you can quickly calculate the edit distance between them and the word with a mistake. To narrow down the circle one may sort by edit distance and leave no more than 15 candidates with edit distance not more than 2. If there are no such variants, it means that the word is new, no corrections are needed, it is simply not in the dictionary.

7. Now I used ngrams to select the best candidate so that the spellchecker will be context sensitive. To do this, I use a sliding window to go through the sentence with the replaced word and search the dictionary for the probability of encountering such a bigram. Here I also apply smoothing for words that are not in the bigram dictionary. For this purpose, the size of unique words in the dictionary is added to the denominator of the probability, as if there is at least one combination of the given word with all other words, and one is added to the numerator. For completely unfamiliar words, I take a probability equal to 1/"size of the dictionary". Then I calculate perplexity for the whole sentence. The sentence with the lowest perplexity had the best candidate for replacement, and I choose exactly it to replace the word with mistake.

## Evaluate on a test set

### Dataset generation

To generate the dataset, it was decided to take 5grams as a basis, like correctly written sentences. For each of them two random words were chosen, each of which will be distorted. Two mistakes are made in each of chosen words. For this purpose, a random error code is generated firstly: 1 - deletion of a random letter, 2 - replacement of a random letter, 3 - addition of a random letter. After that, the selected change is made in the word. In this way sentences with errors corresponding to the correct ones are generated.

In [102]:
def mistakes(sentence):
    """
    Adding distortion to sentence

    :param sentence: sentence to be noised
    :return: distorted sentence
    """
    tokens = RegexpTokenizer(r'\w+').tokenize(sentence)
    sample_list = sample(tokens, 2)
    for sam in sample_list:
        for j in range(2):
            ch = sample(range(1,4), 1)
            if ch == 1:
                # random letter remove
                sentence = sentence.replace(sam, sam.replace(choice(sam), ''))
            elif ch == 2:
                # change random letter
                sentence = sentence.replace(sam, sam.replace(choice(sam),
                                                choice(string.ascii_letters).lower()))
            else:
                # add random letter
                idx = choice(range(len(sam)))
                sam_r = sam[:idx] + choice(string.ascii_letters).lower() + sam[idx:]
                sentence = sentence.replace(sam, sam_r)
    return sentence

To check the accuracy of the algorithm, the following was used: Each word from the correct sentence is compared with each word from the corrected sentence and the proportion of correct words was counted. Then the average of this metric over all sentences in the dataset is calculated. Due to the fact that 2 words out of 5 words in a sentence were distorted, the initial value of this metric is 0.6. An output model that gives this metric a value of ~0.6 does not correct errors or does so incorrectly.

In [103]:
def check(sent_1, sent_2):
    """
    "Accuracy"-like metric for testing sentence

    :param sent_1: correct sentence
    :param sent_2: corrected sentence
    :return: ration of correct words
    """
    correct = 0
    tokenizer = RegexpTokenizer(r'\w+')
    tokens_1 = tokenizer.tokenize(sent_1.lower())
    tokens_2 = tokenizer.tokenize(sent_2.lower())
    for i, j in zip(tokens_1, tokens_2):
        if i == j:
            correct += 1
    return correct/len(tokens_1)

In [129]:
# Generate simple 5gram-based dataset
simple_dataset = []
num = 3000
j = 0
with open('./5gram.txt', 'r', encoding = "ISO-8859-1") as f:
    text = f.read().split('\n')
    shuffle(text)
    for j in tqdm(range(num if num < len(text) else len(text))):
        _, c, d, e, f, g = text[j].split('\t')
        sentence = " ".join([c, d, e, f, g]).lower()
        simple_dataset.append([sentence, mistakes(sentence)])

100%|██████████| 3000/3000 [00:00<00:00, 17578.29it/s]


### Evaluation

I run dataset on both my and Norvig's spellcheckers [4] and measure time and 'accuracy'.

In [137]:
# My SpellCheker evaluation
sc_accuracy = 0
for i in tqdm(range(num)):
    sent = simple_dataset[i]
    correct = sent[0]
    mistake = sent[1]
    changed = sc.spell_check(mistake)
    metric = check(correct, changed)
    sc_accuracy += metric
sc_accuracy /= num
sc_accuracy

100%|██████████| 3000/3000 [02:35<00:00, 19.35it/s]
0.9238714285714308


In [141]:
# Norvig SpellChecker evaluation
norvig_accuracy = 0
for i in tqdm(range(num)):
    sent = simple_dataset[i]
    correct = sent[0]
    mistake = sent[1]
    changed = norvig.full_correct(mistake)
    metric = check(correct, changed)
    norvig_accuracy += metric
norvig_accuracy /= num
norvig_accuracy

100%|██████████| 3000/3000 [02:32<00:00, 19.74it/s]
0.8764269841269858


### Conclusion:
Comparing the result of my spellchecker with Norvig's solution I concluded that my spellchecker is more accurate (0.924) than Norvig's (0.876), although Norvig's spellchecker is slightly faster. This is due to the fact that I have to check the ngram set, while having context sensetive solution. So, in general I can conclude that my spellchecker performed very well!

## References:

[1] - https://norvig.com/ngrams/

[2] - https://github.com/spotify/annoy

[3] - https://wisdomml.in/edit-distance-based-spell-corrector-in-nlp/#:~:text=Edit%20distance%20is%20a%20method,similar%20the%20two%20words%20are.

[4] - https://norvig.com/spell-correct.html